<a href="https://colab.research.google.com/github/liuchiente/Digital_Image_Process_at_NCHU/blob/main/%E7%94%A8Open_CV%E8%87%AA%E5%8B%95%E6%89%BE%E5%87%BAImage%E7%9A%84bounding_box%E4%B8%A6%E8%BD%89%E6%88%90Yolo_Label_Using_OpenCV_to_auto_label_images_for_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境初始化


1.   切換到工作目錄
2.   載入會用到的工具函式
3.   準備資料集

In [ ]:
#掛載Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#載入框架和工具函式
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow

In [ ]:
#切換預設目錄
SYS_DIR = "/content/drive/My Drive/Colab Notebooks/Yololo"
if not os.path.isdir(SYS_DIR):
    os.mkdir(SYS_DIR)
os.chdir(SYS_DIR)

## 準備資料集


本次準備的是從Kaggle上取得的腐爛、新鮮水果資料集, 資料集包含蘋果、香蕉、橘子等腐爛和新鮮的對比,詳情可見Kaggle網址: https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification


In [ ]:
#為了能取得Kaggle資料及,需先下載Kagle套件
! pip install -q kaggle

In [ ]:
# 下載資料集
! kaggle datasets download 'sriramr/fruits-fresh-and-rotten-for-classification'

100% 3.58G/3.58G [00:56<00:00, 112MB/s]
100% 3.58G/3.58G [00:57<00:00, 67.2MB/s]


In [ ]:
# 解壓縮資料集
!unzip fruits-fresh-and-rotten-for-classification.zip

## 資料預先處理
因為這批資料起碼有一萬張以上圖片,若逐張標記的話可能要花很多時間,所以使用Open CV來做批次處理。 

1.   確認圖片有沒有異常
2.   對圖片進行標記,劃出區域

In [ ]:
# 定義找出bounding box
def get_imgage_range(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  lower = np.array([0, 0, 0], dtype = "uint8")
  upper = np.array([255, 50, 255], dtype = "uint8")
  img = cv2.inRange(img, lower, upper)
  img = cv2.blur(img, (2, 2))
  ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)
  img = cv2.bitwise_not(img)
  contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = max(contours, key=lambda x: cv2.contourArea(x))
  #取得方框位置
  x, y, w, h = cv2.boundingRect(contours)
  
  xmin=x
  ymin=y
  xmax=x+w
  ymax=y+h

  #找出圖片尺寸
  dimensions = img.shape
  hi = img.shape[0]
  wi = img.shape[1]
  
  #計算Open CV所需要的比例,劃出bounding box
  xp = (xmin + (xmax-xmin)/2) * 1.0 / wi
  yp = (ymin + (ymax-ymin)/2) * 1.0 / hi
  wp = (xmax-xmin) * 1.0 / wi
  hp = (ymax-ymin) * 1.0 / hi
  return xp, yp, wp, hp

In [ ]:
# 首先清除Yolo訓練時需要的Train、Test文字紀錄檔
if os.path.exists('./darknet/rot/train.txt'):
  os.remove('./darknet/rot/train.txt')
if os.path.exists('./darknet/rot/text.txt'):
  os.remove('./darknet/rot/text.txt')

In [ ]:
#重新訓練或是一開始訓練時,我們會先刪除原本的標籤檔案
for dirname, _, filenames in os.walk('./darknet/rot/dataset'):
    counter=0
    for filename in  glob.iglob(dirname+'/*.txt') :
      os.remove(filename)      

In [ ]:
#確認是不是有圖片檔案路徑異常,造成沒辦法讀取到圖片
for dirname, _, filenames in os.walk('./darknet/rot/dataset'):
    for img_path in  glob.iglob(dirname+'/*.png') :
      if not os.path.exists(img_path):
        print(img_path)
        os.remove(img_path) 

In [ ]:
#開始遍歷資料夾裡面的圖片,並為它加上標籤
imgcnt=0
for dirname, _, filenames in os.walk('./darknet/rot/dataset'):
    for img_path in  glob.iglob(dirname+'/*.png') :

        lbl_path=os.path.splitext(img_path)[0]+".txt"
        dirnames=dirname.split('/')
        if len(dirnames)>4:
          #定義標籤
          label=0
          if dirnames[5] == 'freshapples':
            label=0
          elif dirnames[5] == 'freshbanana':
            label=1
          elif dirnames[5] == 'freshoranges':
            label=2
          elif dirnames[5] == 'rottenapples':
            label=3
          elif dirnames[5] == 'rottenbanana':
            label=4
          elif dirnames[5] == 'rottenoranges':
            label=5
          
          #轉換轉換Yolo比例
          if os.path.exists(img_path):
              #取得標籤
              xmin, ymin, xmax, ymax=get_imgage_range(img_path)
              #儲存標籤
              with open(lbl_path, 'a+') as f:
                f.write('%s %s %s %s %s\n' % (label,xmin, ymin, xmax, ymax))
              #紀錄Train、Text路徑
              with open('./darknet/rot/'+dirnames[4]+ '.txt', 'a+') as f:
                line_txt = [img_path, '\n']
                f.writelines(line_txt)
              imgcnt=imgcnt+1
          else:
            print(img_path)

In [ ]:
print(str(imgcnt)+" images has been processd")

13319


In [ ]:
!ls